<a href="https://colab.research.google.com/github/ClouDevops101/size-limit/blob/gh-pages/recommended-engine-with-udemy-data-v1.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# build a online courses recommendation system

In [ ]:
# importing libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Load the data 
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [2]:
#Store the data 
df = pd.read_csv('udemy-data.csv')
# show the first 3 rows of data 
df.head(3)

,Course_id,title,description,headline,category,subcategory,instructors_name,instructors_job_title,price,avg_rating,num_reviews,created
0,1070968,Ultimate Investment Banking Course,<p>Our dedicated approach and solid methodolog...,Master all the Financial Markets concepts : St...,Finance & Accounting,Finance,VBA Finance,Trainer,€94.99,4.100000,99,2017-01-11T03:06:21Z
1,1070976,"Complete E-Commerce Course - Java,Spring,Hiber...",<p><strong></strong></p> <p><strong>WELCOME T...,Step-by-step guide on how to develop a complet...,Development,Web Development,Le Deng,"ldeng.net/courses, consultant, thousands of st...",€94.99,4.708334,730,2017-01-11T03:16:09Z
2,1071018,FL Studio Sytrus Sound Design Course,<p>In this course Alex Merced will teach you a...,Making Sounds with one of the best FM Synths o...,Music,Music Production,Alex Merced,Knowledge is Freedom,Free,3.950000,327,2017-01-11T04:11:35Z


In [3]:
df.shape


(914, 12)

In [4]:
# Creat a list of important columns for the recommendation engine
columns= ['title','description','headline','category','subcategory','instructors_name','instructors_job_title']

In [5]:
# show the data 
df[columns].head(3)


,title,description,headline,category,subcategory,instructors_name,instructors_job_title
0,Ultimate Investment Banking Course,<p>Our dedicated approach and solid methodolog...,Master all the Financial Markets concepts : St...,Finance & Accounting,Finance,VBA Finance,Trainer
1,"Complete E-Commerce Course - Java,Spring,Hiber...",<p><strong></strong></p> <p><strong>WELCOME T...,Step-by-step guide on how to develop a complet...,Development,Web Development,Le Deng,"ldeng.net/courses, consultant, thousands of st..."
2,FL Studio Sytrus Sound Design Course,<p>In this course Alex Merced will teach you a...,Making Sounds with one of the best FM Synths o...,Music,Music Production,Alex Merced,Knowledge is Freedom


In [6]:
# check for missing values in the important columns
df[columns].isnull().values.any()

# data cleaning is needed

True

In [7]:
# Create a function to combine the values of the important columns into a single string
def get_important_features(data):
  important_features = []
  for i in range(0, data.shape[0]):
    important_features.append(str(data['title'][i])+' '+str(data['description'][i])+' '+str(data['headline'][i])+' '+str(data['category'][i])+' '+str(data['subcategory'][i])+' '+str(data['instructors_name'][i])+' '+str(data['instructors_job_title'][i]) )
  return important_features


In [103]:
# Create a column to hold the combined strings
df['important_features'] = get_important_features(df)

#show the data 
df.head(3)

,Course_id,title,description,headline,category,subcategory,instructors_name,instructors_job_title,price,avg_rating,num_reviews,created,important_features
0,1070968,Ultimate Investment Banking Course,<p>Our dedicated approach and solid methodolog...,Master all the Financial Markets concepts : St...,Finance & Accounting,Finance,VBA Finance,Trainer,€94.99,4.100000,99,2017-01-11T03:06:21Z,Ultimate Investment Banking Course <p>Our dedi...
1,1070976,"Complete E-Commerce Course - Java,Spring,Hiber...",<p><strong></strong></p> <p><strong>WELCOME T...,Step-by-step guide on how to develop a complet...,Development,Web Development,Le Deng,"ldeng.net/courses, consultant, thousands of st...",€94.99,4.708334,730,2017-01-11T03:16:09Z,"Complete E-Commerce Course - Java,Spring,Hiber..."
2,1071018,FL Studio Sytrus Sound Design Course,<p>In this course Alex Merced will teach you a...,Making Sounds with one of the best FM Synths o...,Music,Music Production,Alex Merced,Knowledge is Freedom,Free,3.950000,327,2017-01-11T04:11:35Z,FL Studio Sytrus Sound Design Course <p>In thi...


In [14]:
# creating stop word
stop_words = stopwords.words('english') + stopwords.words('french') + stopwords.words('spanish') + stopwords.words('dutch') 
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [106]:
# convert the text to a matrix of token counts 
df['important_features'].replace('<[^<]+?>',' ',regex=True, inplace = True)
print (df['important_features'].head(3))
df['important_features'] = df['important_features'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
cm = CountVectorizer().fit_transform(df['important_features'])

# GEt the cosine similarity matrix from the count matrix
cs = cosine_similarity(cm)
#Print the cosine similarity matrix
print(cs)


0    Ultimate Investment Banking Course Our dedicat...
1    Complete E-Commerce Course - Java,Spring,Hiber...
2    FL Studio Sytrus Sound Design Course In course...
Name: important_features, dtype: object
[[1.         0.14385183 0.12341645 ... 0.07802535 0.00183064 0.10052742]
 [0.14385183 1.         0.15303305 ... 0.15528265 0.07014129 0.10158148]
 [0.12341645 0.15303305 1.         ... 0.09658793 0.00453232 0.05943563]
 ...
 [0.07802535 0.15528265 0.09658793 ... 1.         0.00354591 0.06393776]
 [0.00183064 0.07014129 0.00453232 ... 0.00354591 1.         0.        ]
 [0.10052742 0.10158148 0.05943563 ... 0.06393776 0.         1.        ]]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [107]:
# Get the shape of the cosine similarity matrix 
cs.shape


(914, 914)

In [118]:
# Get the title of the course that the user likes 
title_to_search = 'Linux for Absolute Beginners!'

# Find the course id
course_id = df[df.title == title_to_search]['Course_id'].index
print (course_id[0])
#course_id = 251

558


In [119]:
# Create a list of enumeration for the similarity score [(course_id, similiraty score ),(..)]
scores = list(enumerate(cs[course_id[0]]))

In [120]:
#sort the list 
sorted_scores = sorted(scores, key = lambda x:x[1], reverse  = True)
sorted_scores = sorted_scores[1:]


In [121]:
#Print the sorted scores
print(sorted_scores)

[(627, 0.4696293974394789), (681, 0.46299992260714473), (309, 0.45721565504361844), (146, 0.4548301348344854), (329, 0.4216959954814239), (290, 0.40914437815773336), (834, 0.3997129746406082), (17, 0.39780063304227814), (480, 0.37193893517799054), (823, 0.3707205584466861), (77, 0.36947545987554586), (320, 0.3691782302769622), (372, 0.36881149777098465), (849, 0.3680615931414359), (91, 0.3615454172058361), (579, 0.35902129898909274), (719, 0.3569539806574639), (423, 0.3553807358720963), (88, 0.3521591772713783), (220, 0.35152772320691295), (433, 0.3454881803282798), (520, 0.3433376258298254), (141, 0.34298673460924484), (1, 0.3424797112692641), (839, 0.3381490450531511), (289, 0.3372038865217675), (589, 0.3355692651136668), (122, 0.33334956547460576), (399, 0.3325908951159447), (732, 0.3320820126787287), (487, 0.3299168986395675), (696, 0.32946947440739094), (899, 0.3259150606293811), (214, 0.32296077883625995), (734, 0.32295246832377883), (519, 0.3224993150246893), (124, 0.32110711767

In [124]:
# Create a loop to print the first 7 similar movies
j = 0
print('The 7 most recommended courses : ', title_to_search, 'are:\n')
print('ID  Course Title' )
for item in sorted_scores:
  course_title = df[df.index == item[0]]['title'].values[0]
  course_id = df[df.index == item[0]]['Course_id'].values[0]
  course_headline = df[df.index == item[0]]['headline'].values[0]
  print(course_id, course_title, " | " + course_headline)
  j=j+1
  if j>10:
    break

The 7 most recommended courses :  Linux for Absolute Beginners! are:

ID  Course Title
1100052 Linux Terminal for beginners  | Learn Linux and Supercharge Your IT Career
1102606 Quickbooks Basic to Advance Training Course in Hindi Urdu  | Quickbooks Basic to Advance Training Course in Hindi Urdu Covers all Popular Quickbooks Versions with  pro enterprise
1085644 Ubuntu Linux for beginners  | Introduction to Ubuntu Linux
1078254 Tally Basic to Advance Training in Hindi / Urdu 2021  | Tally.ERP 9 Expert course for 2021 Updated | include Tally Payroll | Tally Accounting and Inventory
1087426 The Complete iOS 11 Developer Course - Beginner To Advanced  | In the complete iOS 11 Developer Course students will learn Swift4 creating real apps like Instagram & Foursquare clones
1084874 Subnetting - Become a Subnet Master!  | Easy IPv4 Subnetting for CCNA and Industry! You will become a Subnet Master Quickly!
1109478 SAP GTS Training  | SAP Global Trade System - Business scenarios and configurat